
# Análisis de microbiota con grafos esparsos y dinámica de osciladores

Este cuaderno desarrolla un pipeline completo para analizar datos de microbiota **sin submuestreo**.  El objetivo es clusterizar las muestras usando distintas vistas de similitud (diversidad alfa, CLR–coseno, PPMI) y fusionarlas en un grafo esparso, sobre el cual se simula la dinámica de osciladores de **Kuramoto** para obtener una afinidad de fase y detectar comunidades.

**Requisitos clave:**

- Usar **todas** las OTUs y **todas** las muestras disponibles.
- Evitar densificar matrices de grafos (no usar `.toarray()` sobre grafos grandes).
- Automatizar la detección del archivo OTU y manejar formatos/duplicados.
- Construir las vistas de similitud (alfa, CLR, PPMI) y fusionarlas con pesos.
- Implementar Kuramoto con integrador Heun/RK2 y afinidad de fase en bloques.
- Clusterizar con **Louvain** cuando sea posible; usar **Label Propagation** en caso de error, y **Leiden** opcionalmente.
- Generar visualizaciones y un análisis de estabilidad basado en NMI.


In [ ]:

# Bloque 1: Configuración general
import sys
import subprocess
import numpy as np
import pandas as pd
import scipy.sparse as sp
import math
import random
import os
from pathlib import Path
import matplotlib.pyplot as plt

# Función para instalar paquetes silenciosamente
def silent_install(package):
    try:
        __import__(package if package != 'python-louvain' else 'community')
        print(f"{package} ya está instalado.")
        return True
    except Exception:
        try:
            print(f"Instalando {package}…")
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', package, '--quiet'])
            print(f"{package} instalado correctamente.")
            return True
        except Exception as e:
            print(f"No se pudo instalar {package}: {e}")
            return False

# Instalar paquetes requeridos
for pkg in ['networkx', 'python-louvain']:
    silent_install(pkg)
# Instalar opcionales
tmp = silent_install('python-igraph')
if tmp:
    silent_install('leidenalg')

# Importar después de instalación
try:
    import networkx as nx
    # Verificar si networkx tiene la función from_scipy_sparse_matrix; en versiones mínimas puede faltar
    if not hasattr(nx, 'from_scipy_sparse_matrix'):
        print('NetworkX no soporta from_scipy_sparse_matrix; no se usará.')
        nx = None
except Exception:
    nx = None
try:
    import community as community_louvain
except Exception:
    community_louvain = None
try:
    import igraph as ig
    import leidenalg
except Exception:
    ig = None
    leidenalg = None
from sklearn.metrics import normalized_mutual_info_score

# Semillas y rutas
GLOBAL_SEED = 42
np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
PROJECT_ROOT = Path('.').resolve()
DATA_DIR = PROJECT_ROOT / 'Datos'
RESULTS_DIR = PROJECT_ROOT / 'Resultados'
RESULTS_DIR.mkdir(exist_ok=True)

# Parámetros por defecto
TOPK = 12
weights = {'alpha': 0.35, 'ppmi': 0.45, 'clr': 0.20}
Kg_default_vals = np.geomspace(0.7, 1.8, 6)
kuramoto_defaults = dict(dt=0.02, T=1400, burn=500, alpha=0.18, seed=7, normalize_by_degree=True)
phase_affinity_defaults = dict(stride=3, chunk_n=256, use_float32=True)
ANONYMIZE_IDS = False

print(f"Proyecto inicializado en {PROJECT_ROOT}")
print(f"Datos en {DATA_DIR}")
print(f"Resultados en {RESULTS_DIR}")


In [ ]:

# Bloque 2: Autodescubrimiento del archivo OTU
from pathlib import Path

def guess_otu_path(data_dir: Path):
    # Buscar archivo con extensión .otus
    for file in data_dir.iterdir():
        if file.is_file() and file.suffix.lower() == '.otus':
            return file, False
    # Buscar archivos que contengan 'otu'
    candidates = []
    for file in data_dir.iterdir():
        if file.is_file() and 'otu' in file.stem.lower() and file.suffix.lower() in {'.tsv', '.csv', '.txt'}:
            candidates.append(file)
    if candidates:
        return candidates[0], False
    # Generar sintético
    print("No se encontró archivo OTU; generando datos sintéticos…")
    n_otus, n_samples = 50, 20
    rng = np.random.default_rng(GLOBAL_SEED)
    synthetic_counts = rng.poisson(lam=5, size=(n_otus, n_samples))
    synthetic_path = data_dir / 'synthetic.otus'
    synthetic_df = pd.DataFrame(synthetic_counts, columns=[f'sample_{i:03d}' for i in range(n_samples)])
    synthetic_df.insert(0, 'OTU_ID', [f'OTU_{i:03d}' for i in range(n_otus)])
    synthetic_df.to_csv(synthetic_path, sep='	', index=False)
    return synthetic_path, True

otu_path, synthetic_data = guess_otu_path(DATA_DIR)
print(f"Archivo OTU detectado: {otu_path.name} (sintético={synthetic_data})")


In [3]:

# Bloque 3: Lectura robusta del OTU
import pandas as pd
import numpy as np
import scipy.sparse as sp


def load_otu_data(otu_file: Path):
    # Detectar delimitador
    with open(otu_file, 'rb') as f:
        header = f.readline().decode('utf-8', errors='ignore')
        if ',' in header and header.count(',') > header.count('	'):
            delim = ','
        elif '	' in header:
            delim = '	'
        else:
            delim = None
    try:
        df = pd.read_csv(otu_file, sep=delim, engine='python')
    except Exception:
        df = pd.read_csv(otu_file, sep=None, engine='python')
    # Quitar columna taxonomy si existe
    for col in df.columns:
        if col.lower() in {'taxonomy','taxonomyid','tax'}:
            df = df.drop(columns=[col])
            break
    first_col = df.columns[0]
    first_vals = df[first_col].astype(str).values
    def looks_like_sample_id(x):
        return any(c.isalpha() for c in x) and '_' in x
    orientation = 'samples_as_rows' if all(looks_like_sample_id(str(v)) for v in first_vals[:min(len(first_vals), 10)]) else 'otus_as_rows'
    if orientation == 'samples_as_rows':
        sample_ids = df[first_col].astype(str).tolist()
        otu_df = df.drop(columns=[first_col])
        otu_ids = otu_df.columns.astype(str).tolist()
        counts = otu_df.values.astype(float).T
    else:
        otu_ids = df[first_col].astype(str).tolist()
        otu_df = df.drop(columns=[first_col])
        sample_ids = otu_df.columns.astype(str).tolist()
        counts = otu_df.values.astype(float)
    # Agrupar OTUs duplicadas
    if len(set(otu_ids)) != len(otu_ids):
        temp_df = pd.DataFrame(counts, index=otu_ids, columns=sample_ids)
        temp_df = temp_df.groupby(level=0).sum()
        otu_ids = temp_df.index.astype(str).tolist()
        counts = temp_df.values.astype(float)
    # Eliminar OTUs vacías
    counts_sum = counts.sum(axis=1)
    mask_nonzero = counts_sum > 0
    counts = counts[mask_nonzero]
    otu_ids = [otu_ids[i] for i, flag in enumerate(mask_nonzero) if flag]
    # Unicidad de sample_ids
    seen = {}
    uniq_sample_ids = []
    for sid in sample_ids:
        if sid not in seen:
            seen[sid] = 1
            uniq_sample_ids.append(sid)
        else:
            seen[sid] += 1
            uniq_sample_ids.append(f"{sid}_{seen[sid]}")
    sample_ids = uniq_sample_ids
    counts_matrix = sp.csr_matrix(counts)
    print(f"OTU cargado: {counts_matrix.shape[0]} OTUs x {counts_matrix.shape[1]} muestras")
    return counts_matrix, otu_ids, sample_ids

counts_matrix, otu_ids, sample_ids = load_otu_data(otu_path)


OTU cargado: 4720 OTUs x 441 muestras


In [4]:

# Bloque 4: Métricas alfa y similitud
import numpy as np
import pandas as pd


def compute_alpha_metrics(counts: sp.csr_matrix, otu_ids, sample_ids):
    X = counts.toarray()
    X_samp = X.T
    richness = (X_samp > 0).sum(axis=1)
    row_sums = X_samp.sum(axis=1)
    ps = X_samp + 1e-12
    ps = ps / row_sums[:, None]
    shannon = -np.sum(ps * np.log(ps + 1e-16), axis=1)
    simpson = 1.0 - np.sum(ps**2, axis=1)
    alpha_df = pd.DataFrame({'sample_id': sample_ids, 'richness': richness, 'shannon': shannon, 'simpson': simpson})
    for col in ['richness','shannon','simpson']:
        m = alpha_df[col].mean()
        s = alpha_df[col].std(ddof=1)
        alpha_df[col+'_z'] = (alpha_df[col] - m) / (s + 1e-8)
    X_z = alpha_df[['richness_z','shannon_z','simpson_z']].values
    norms = np.linalg.norm(X_z, axis=1)
    norms[norms==0] = 1.0
    X_norm = X_z / norms[:,None]
    sim_alpha = X_norm @ X_norm.T
    np.fill_diagonal(sim_alpha,0.0)
    return sim_alpha, alpha_df

sim_alpha, alpha_df = compute_alpha_metrics(counts_matrix, otu_ids, sample_ids)
print("Matriz de similitud alfa calculada.")


Matriz de similitud alfa calculada.


In [5]:

# Bloque 5: Cálculo de CLR y PPMI
import numpy as np
import scipy.sparse as sp


def compute_clr_similarity(counts: sp.csr_matrix):
    X = counts.toarray()
    X_pseudo = X + 1.0
    prop = X_pseudo / X_pseudo.sum(axis=0, keepdims=True)
    log_prop = np.log(prop)
    clr = log_prop - log_prop.mean(axis=0, keepdims=True)
    Xclr = clr.T
    norms = np.linalg.norm(Xclr, axis=1)
    norms[norms==0] = 1.0
    Xclr_norm = Xclr / norms[:,None]
    sim_clr = Xclr_norm @ Xclr_norm.T
    np.fill_diagonal(sim_clr,0.0)
    return sim_clr

def compute_ppmi_similarity(counts: sp.csr_matrix):
    X = counts.astype(float)
    total = X.sum()
    row_sum = np.array(X.sum(axis=1)).flatten()
    col_sum = np.array(X.sum(axis=0)).flatten()
    X_coo = X.tocoo()
    expected = (row_sum[X_coo.row] * col_sum[X_coo.col]) / total
    with np.errstate(divide='ignore', invalid='ignore'):
        ppmi_data = np.log((X_coo.data * total) / expected)
    ppmi_data[~np.isfinite(ppmi_data)] = 0.0
    ppmi_data = np.maximum(0.0, ppmi_data)
    ppmi_matrix = sp.csr_matrix((ppmi_data, (X_coo.row, X_coo.col)), shape=X.shape)
    M = (ppmi_matrix.T @ ppmi_matrix).toarray()
    norms = np.sqrt(np.diag(M))
    norms[norms==0] = 1.0
    sim_ppmi = M / (norms[:,None] * norms[None,:])
    np.fill_diagonal(sim_ppmi,0.0)
    return sim_ppmi

sim_clr = compute_clr_similarity(counts_matrix)
print("Similitud CLR calculada.")
sim_ppmi = compute_ppmi_similarity(counts_matrix)
print("Similitud PPMI calculada.")


Similitud CLR calculada.
Similitud PPMI calculada.


In [6]:

# Bloque 6: Fusión de vistas y grafo final
import scipy.sparse as sp
import numpy as np


def build_knn_graph_from_dense(similarity: np.ndarray, topk: int, mutual: bool=False):
    n = similarity.shape[0]
    neighbor_list=[]
    weight_list=[]
    for i in range(n):
        row = similarity[i].copy()
        row[i] = -np.inf
        if topk < n:
            idxs = np.argpartition(row, -topk)[-topk:]
        else:
            idxs = np.arange(n)
        idxs = idxs[np.argsort(row[idxs])[::-1]]
        valid = row[idxs] > 0
        idxs = idxs[valid]
        weights = row[idxs]
        neighbor_list.append(idxs)
        weight_list.append(weights)
    rows=[]
    cols=[]
    data=[]
    for i in range(n):
        for j,w in zip(neighbor_list[i], weight_list[i]):
            if mutual:
                if i in neighbor_list[j]:
                    rows.append(i); cols.append(j); data.append(w)
            else:
                rows.append(i); cols.append(j); data.append(w)
    W = sp.coo_matrix((data,(rows,cols)), shape=(n,n))
    W_sym = W.tocsr().maximum(W.tocsr().transpose())
    return W_sym

def sparse_topk(W: sp.csr_matrix, k:int, mutual: bool=True):
    n = W.shape[0]
    rows=[]; cols=[]; data=[]
    for i in range(n):
        start=W.indptr[i]; end=W.indptr[i+1]
        row_indices=W.indices[start:end]
        row_data=W.data[start:end]
        if len(row_data)==0:
            continue
        if k < len(row_data):
            idxs=np.argpartition(row_data, -k)[-k:]
        else:
            idxs=np.arange(len(row_data))
        idxs = idxs[np.argsort(row_data[idxs])[::-1]]
        sel_indices = row_indices[idxs]
        sel_data = row_data[idxs]
        for j,w in zip(sel_indices, sel_data):
            rows.append(i); cols.append(j); data.append(w)
    Wk = sp.coo_matrix((data,(rows,cols)), shape=W.shape).tocsr()
    if mutual:
        mask = Wk.multiply(Wk.T)
        W_mutual = Wk.multiply(mask>0)
        W_mutual = W_mutual + W_mutual.T
        W_mutual = W_mutual.maximum(W_mutual.T)
        return W_mutual.tocsr()
    else:
        return Wk.maximum(Wk.T).tocsr()

# Construir vistas y fusionar
W_alpha = build_knn_graph_from_dense(sim_alpha, topk=TOPK, mutual=False)
if W_alpha.nnz>0:
    W_alpha_norm = W_alpha.copy(); m = W_alpha_norm.data.max();
    if m>0: W_alpha_norm.data /= m
else:
    W_alpha_norm = W_alpha
W_ppmi = build_knn_graph_from_dense(sim_ppmi, topk=TOPK, mutual=False)
if W_ppmi.nnz>0:
    W_ppmi_norm = W_ppmi.copy(); m = W_ppmi_norm.data.max();
    if m>0: W_ppmi_norm.data /= m
else:
    W_ppmi_norm = W_ppmi
W_clr = build_knn_graph_from_dense(sim_clr, topk=TOPK, mutual=False)
if W_clr.nnz>0:
    W_clr_norm = W_clr.copy(); m = W_clr_norm.data.max();
    if m>0: W_clr_norm.data /= m
else:
    W_clr_norm = W_clr

W_fused = None
for name, Wv in [('alpha', W_alpha_norm),('ppmi',W_ppmi_norm),('clr',W_clr_norm)]:
    weight = weights.get(name,0.0)
    if Wv is not None and Wv.nnz>0 and weight>0:
        W_scaled = Wv.multiply(weight)
        if W_fused is None:
            W_fused = W_scaled.tocsr()
        else:
            W_fused = (W_fused + W_scaled).tocsr()
if W_fused is None:
    n = len(sample_ids)
    W_fused = sp.csr_matrix((n,n))
W_fused_knn = sparse_topk(W_fused, k=TOPK, mutual=True)
print(f"Grafo fusionado: forma {W_fused_knn.shape}, nnz = {W_fused_knn.nnz}")


Grafo fusionado: forma (441, 441), nnz = 3622


In [7]:

# Bloque 7: Kuramoto y afinidad de fase
import numpy as np
import scipy.sparse as sp
import math


def simulate_kuramoto(W: sp.csr_matrix, Kg: float, dt: float, T: int, burn: int, alpha: float=0.18, seed:int=7, normalize_by_degree:bool=True):
    n = W.shape[0]
    rng = np.random.default_rng(seed)
    theta = rng.uniform(0.0, 2.0 * math.pi, size=n)
    natural = rng.normal(loc=0.0, scale=1.0, size=n)
    neighbors=[]; weights_list=[]; degrees=np.array(W.sum(axis=1)).flatten()
    for i in range(n):
        start=W.indptr[i]; end=W.indptr[i+1]
        neighbors.append(W.indices[start:end])
        weights_list.append(W.data[start:end])
    phases=[]
    for step in range(T):
        dtheta = np.zeros(n, dtype=float)
        for i in range(n):
            if len(neighbors[i])==0: continue
            diffs = theta[neighbors[i]] - theta[i]
            if normalize_by_degree:
                denom = degrees[i] if degrees[i]>0 else 1.0
                coupling = Kg / denom
            else:
                coupling = Kg
            dtheta[i] = natural[i] + coupling * np.sum(weights_list[i] * np.sin(diffs))
        k1 = dtheta
        theta_pred = theta + dt * k1
        dtheta2 = np.zeros(n, dtype=float)
        for i in range(n):
            if len(neighbors[i])==0: continue
            diffs2 = theta_pred[neighbors[i]] - theta_pred[i]
            if normalize_by_degree:
                denom = degrees[i] if degrees[i]>0 else 1.0
                coupling = Kg / denom
            else:
                coupling = Kg
            dtheta2[i] = natural[i] + coupling * np.sum(weights_list[i] * np.sin(diffs2))
        theta = theta + dt * dtheta2
        if step >= burn:
            phases.append(theta.copy())
    return np.array(phases)

def phase_affinity(phases: np.ndarray, stride:int=3, chunk_n:int=256, use_float32:bool=True):
    dtype = np.float32 if use_float32 else np.float64
    phases_sub = phases[::stride]
    n_time, n_nodes = phases_sub.shape
    cos_vals = np.cos(phases_sub).astype(dtype)
    sin_vals = np.sin(phases_sub).astype(dtype)
    aff = np.zeros((n_nodes,n_nodes), dtype=dtype)
    for start in range(0,n_nodes,chunk_n):
        end=min(start+chunk_n, n_nodes)
        C = cos_vals.T @ cos_vals[:,start:end]
        S = sin_vals.T @ sin_vals[:,start:end]
        aff[:,start:end] = (C + S) / n_time
    aff = np.maximum(aff, aff.T)
    np.fill_diagonal(aff,0.0)
    return aff

Kg_example = Kg_default_vals[0]
print(f"Simulando Kuramoto (K_g={Kg_example}) para ejemplo…")
phases_example = simulate_kuramoto(W_fused_knn, Kg=Kg_example, **kuramoto_defaults)
aff_example = phase_affinity(phases_example, **phase_affinity_defaults)
print("Afinidad de fase (ejemplo) calculada.")


Simulando Kuramoto (K_g=0.7) para ejemplo…
Afinidad de fase (ejemplo) calculada.


In [8]:

# Bloque 8: Clustering comunitario
import numpy as np
import scipy.sparse as sp

def _modularity(W: sp.csr_matrix, labels: np.ndarray) -> float:
    """
    Calcular la modularidad de una partición en un grafo no dirigido y ponderado representado por una matriz CSR.
    Si la suma total de pesos es cero o ocurre un error, devuelve -1e9.
    """
    try:
        m = W.data.sum() / 2.0
        if m <= 0:
            return -1e9
        degrees = np.array(W.sum(axis=1)).flatten()
        Q = 0.0
        unique_labels = np.unique(labels)
        for lbl in unique_labels:
            idx = np.where(labels == lbl)[0]
            subgraph = W[idx][:, idx]
            e_c = subgraph.data.sum()
            a_c = degrees[idx].sum()
            Q += (e_c / (2.0 * m)) - (a_c / (2.0 * m)) ** 2
        return Q
    except Exception:
        return -1e9

def _label_propagation_sparse(W: sp.csr_matrix, max_iter: int = 100, seed: int = 7) -> np.ndarray:
    """
    Implementación sencilla de label propagation sobre un grafo esparso no dirigido.
    Devuelve un vector de etiquetas de comunidad para cada nodo.
    """
    np.random.seed(seed)
    n = W.shape[0]
    labels = np.arange(n, dtype=int)
    for iteration in range(max_iter):
        nodes = np.arange(n)
        np.random.shuffle(nodes)
        changed = False
        for i in nodes:
            start = W.indptr[i]
            end = W.indptr[i + 1]
            neighbors = W.indices[start:end]
            weights = W.data[start:end]
            if len(neighbors) == 0:
                continue
            label_weights = {}
            for j, w in zip(neighbors, weights):
                lbl = labels[j]
                label_weights[lbl] = label_weights.get(lbl, 0.0) + w
            max_w = max(label_weights.values())
            best_labels = [lbl for lbl, w in label_weights.items() if w == max_w]
            new_label = min(best_labels)
            if new_label != labels[i]:
                labels[i] = new_label
                changed = True
        if not changed:
            break
    unique_labels = np.unique(labels)
    label_map = {old: new for new, old in enumerate(unique_labels)}
    labels = np.array([label_map[lbl] for lbl in labels], dtype=int)
    return labels

def cluster_graph(W: sp.csr_matrix, method_preference=['louvain','label_propagation','leiden'], seed:int=7):
    """
    Agrupa un grafo ponderado esparso. Orden de preferencia: Louvain via python-louvain, Leiden via igraph,
    label propagation de networkx, y finalmente label propagation implementado manualmente. Devuelve
    las etiquetas, la modularidad y el método usado.
    """
    n = W.shape[0]
    if W.nnz == 0:
        return np.zeros(n, dtype=int), -1e9, 'empty'
    for method in method_preference:
        if method == 'louvain' and 'community_louvain' in globals() and community_louvain is not None:
            try:
                if 'nx' in globals() and nx is not None:
                    G = nx.from_scipy_sparse_matrix(W, parallel_edges=False)
                    part = community_louvain.best_partition(G, weight='weight', random_state=seed)
                    labels = np.array([part.get(i, 0) for i in range(n)])
                    Q = community_louvain.modularity(part, G, weight='weight')
                    return labels, Q, 'louvain'
            except Exception as e:
                print(f"Louvain falló: {e}")
                continue
        if method == 'leiden' and 'ig' in globals() and ig is not None and 'leidenalg' in globals() and leidenalg is not None:
            try:
                coo = W.tocoo()
                edges_list = list(zip(coo.row.tolist(), coo.col.tolist()))
                weights_list = coo.data.tolist()
                g_ig = ig.Graph(n=W.shape[0], edges=edges_list, edge_attrs={'weight': weights_list}, directed=False)
                part = leidenalg.find_partition(g_ig, leidenalg.RBConfigurationVertexPartition, weights=weights_list, seed=seed)
                labels = np.array(part.membership)
                Q = g_ig.modularity(list(labels), weights=weights_list)
                return labels, Q, 'leiden'
            except Exception as e:
                print(f"Leiden falló: {e}")
                continue
        if method == 'label_propagation':
            if 'nx' in globals() and nx is not None:
                try:
                    G = nx.from_scipy_sparse_matrix(W, parallel_edges=False)
                    for (u, v, d) in G.edges(data=True):
                        d['weight'] = float(d.get('weight', 1.0))
                    communities = list(nx.algorithms.community.asyn_lpa_communities(G, weight='weight', seed=seed))
                    labels = np.zeros(n, dtype=int)
                    for idx, com in enumerate(communities):
                        for node in com:
                            labels[node] = idx
                    Q = -1e9
                    try:
                        Q = nx.algorithms.community.quality.modularity(G, communities, weight='weight')
                    except Exception:
                        pass
                    return labels, Q, 'label_propagation'
                except Exception as e:
                    print(f"Label Propagation (networkx) falló: {e}")
                    # continue to custom
            try:
                labels = _label_propagation_sparse(W, max_iter=100, seed=seed)
                Q = _modularity(W, labels)
                return labels, Q, 'label_propagation_fallback'
            except Exception as e:
                print(f"Label Propagation (custom) falló: {e}")
                continue
    return np.zeros(n, dtype=int), -1e9, 'fallback'

# Ejemplo de uso de cluster_graph con una matriz de afinidad pequeña
try:
    aff_example = np.array([[1.0, 0.8, 0.2], [0.8, 1.0, 0.3], [0.2, 0.3, 1.0]], dtype=float)
    aff_example = (aff_example + aff_example.T) / 2.0
    W_example = sp.csr_matrix(aff_example)
    labels_example, Q_example, method_example = cluster_graph(W_example, ['louvain','label_propagation','leiden'])
    print(f"Ejemplo de clustering: método={method_example}, Q={Q_example:.4f}, nº clusters={len(set(labels_example))}")
except Exception as e:
    print(f"No se pudo ejecutar el ejemplo de clustering: {e}")


Ejemplo de clustering: método=label_propagation_fallback, Q=0.1958, nº clusters=3


In [9]:

# Bloque 9: Ejecución del experimento completo
import pandas as pd


def run_experiment_full(counts_matrix: sp.csr_matrix, otu_ids, sample_ids,
                        Kg_vals=None, topk:int=TOPK,
                        weights_dict=None,
                        kuramoto_kwargs=None,
                        phase_affinity_kwargs=None,
                        topk_aff:int=8,
                        method_preference=['louvain','label_propagation','leiden'],
                        anonymize_ids:bool=ANONYMIZE_IDS):
    if Kg_vals is None:
        Kg_vals = Kg_default_vals
    if weights_dict is None:
        weights_dict = weights
    if kuramoto_kwargs is None:
        kuramoto_kwargs = kuramoto_defaults.copy()
    if phase_affinity_kwargs is None:
        phase_affinity_kwargs = phase_affinity_defaults.copy()
    # Recalcular vistas
    sim_alpha_local, alpha_df_local = compute_alpha_metrics(counts_matrix, otu_ids, sample_ids)
    W_alpha_local = build_knn_graph_from_dense(sim_alpha_local, topk=topk, mutual=False)
    if W_alpha_local.nnz>0:
        W_alpha_norm = W_alpha_local.copy(); m=W_alpha_norm.data.max();
        if m>0: W_alpha_norm.data /= m
    else:
        W_alpha_norm = W_alpha_local
    sim_clr_local = compute_clr_similarity(counts_matrix)
    W_clr_local = build_knn_graph_from_dense(sim_clr_local, topk=topk, mutual=False)
    if W_clr_local.nnz>0:
        W_clr_norm = W_clr_local.copy(); m=W_clr_norm.data.max();
        if m>0: W_clr_norm.data /= m
    else:
        W_clr_norm = W_clr_local
    sim_ppmi_local = compute_ppmi_similarity(counts_matrix)
    W_ppmi_local = build_knn_graph_from_dense(sim_ppmi_local, topk=topk, mutual=False)
    if W_ppmi_local.nnz>0:
        W_ppmi_norm = W_ppmi_local.copy(); m=W_ppmi_norm.data.max();
        if m>0: W_ppmi_norm.data /= m
    else:
        W_ppmi_norm = W_ppmi_local
    W_fused_local = None
    for name,Wv in [('alpha',W_alpha_norm),('ppmi',W_ppmi_norm),('clr',W_clr_norm)]:
        weight=weights_dict.get(name,0.0)
        if Wv is not None and Wv.nnz>0 and weight>0:
            W_scaled=Wv.multiply(weight)
            if W_fused_local is None:
                W_fused_local = W_scaled.tocsr()
            else:
                W_fused_local = (W_fused_local + W_scaled).tocsr()
    if W_fused_local is None:
        n = counts_matrix.shape[1]
        W_fused_local = sp.csr_matrix((n,n))
    W_fused_knn_local = sparse_topk(W_fused_local, k=topk, mutual=True)
    nnz_fused = W_fused_knn_local.nnz
    density = nnz_fused / (W_fused_knn_local.shape[0]**2)
    print(f"Grafo fusionado: {W_fused_knn_local.shape[0]} nodos, {nnz_fused} aristas, densidad {density:.4e}")
    grid_records=[]
    best_labels=None; best_Q=-np.inf; best_Kg=None; best_method=None
    for Kg in Kg_vals:
        try:
            phases = simulate_kuramoto(W_fused_knn_local, Kg=Kg, **kuramoto_kwargs)
            aff = phase_affinity(phases, **phase_affinity_kwargs)
            W_aff = build_knn_graph_from_dense(aff, topk=topk_aff, mutual=False)
            labels,Q,method_used = cluster_graph(W_aff, method_preference, seed=kuramoto_kwargs.get('seed',7))
            n_clusters=len(set(labels))
            grid_records.append({'Kg':Kg,'Q':Q,'k':n_clusters,'method':method_used})
            print(f"K_g={Kg:.4f}, Q={Q:.4f}, clusters={n_clusters}, método={method_used}")
            if Q > best_Q:
                best_Q=Q; best_labels=labels; best_Kg=Kg; best_method=method_used
        except Exception as e:
            print(f"Error en K_g={Kg}: {e}")
            grid_records.append({'Kg':Kg,'Q':-1e9,'k':1,'method':'error'})
            continue
    grid_df=pd.DataFrame(grid_records)
    if best_labels is None or len(best_labels)!=counts_matrix.shape[1]:
        best_labels=np.zeros(counts_matrix.shape[1],dtype=int)
    output_sample_ids=sample_ids
    id_map_df=None
    if anonymize_ids:
        anon_ids=[f"sample{str(i).zfill(4)}" for i in range(len(sample_ids))]
        id_map_df=pd.DataFrame({'sample_id_original':sample_ids,'sample_id':anon_ids})
        output_sample_ids=anon_ids
    labels_df=pd.DataFrame({'sample_id':output_sample_ids,'cluster':best_labels})
    labels_df.to_csv(RESULTS_DIR/'labels.csv', index=False)
    grid_df.to_csv(RESULTS_DIR/'grid_df.csv', index=False)
    alpha_df_to_save=alpha_df_local.copy()
    if anonymize_ids:
        alpha_df_to_save['sample_id']=output_sample_ids
    alpha_df_to_save.to_csv(RESULTS_DIR/'alpha_metrics.csv', index=False)
    summary_lines=[
        f"Forma de la matriz OTU: {counts_matrix.shape[0]} OTUs x {counts_matrix.shape[1]} muestras",
        f"TOPK de fusión: {topk}",
        f"Número de aristas en el grafo fusionado: {nnz_fused}",
        f"Densidad del grafo fusionado: {density:.4e}",
        f"Mejor K_g: {best_Kg}",
        f"Mejor modularidad Q: {best_Q}",
        f"Número de clústeres: {len(set(best_labels))}",
        f"Método de clustering usado: {best_method}"
    ]
    (RESULTS_DIR/'run_summary.txt').write_text('\n'.join(summary_lines))
    if id_map_df is not None:
        id_map_df.to_csv(RESULTS_DIR/'id_map.csv', index=False)
    return {'labels':best_labels,'labels_df':labels_df,'alpha_df':alpha_df_local,'grid_df':grid_df,'shape_used':counts_matrix.shape,'W_fused':W_fused_knn_local}

results = run_experiment_full(counts_matrix, otu_ids, sample_ids,
                              Kg_vals=Kg_default_vals,
                              topk=TOPK,
                              weights_dict=weights,
                              kuramoto_kwargs=kuramoto_defaults,
                              phase_affinity_kwargs=phase_affinity_defaults,
                              topk_aff=8,
                              method_preference=['louvain','label_propagation','leiden'],
                              anonymize_ids=ANONYMIZE_IDS)
print("Experimento completo finalizado.")


Grafo fusionado: 441 nodos, 3622 aristas, densidad 1.8624e-02


KeyboardInterrupt: 

In [ ]:
# Bloque 10: Visualizaciones
import matplotlib.pyplot as plt
import numpy as np

# --- 10.1 Curva Q vs Kg ---
if not results['grid_df'].empty:
    fig, ax = plt.subplots(figsize=(7,4))
    ax.plot(results['grid_df']['Kg'], results['grid_df']['Q'], marker='o')
    ax.set_xlabel('K_g')
    ax.set_ylabel('Modularidad Q')
    ax.set_title('Curva Q vs K_g')
    ax.grid(True, alpha=0.3)
    fig.tight_layout()
    plt.show()
else:
    print("No hay resultados para graficar Q vs K_g.")

# --- 10.2 Grafo kNN sobre similitud alfa con barras de color ---
try:
    # 1) Construcción del kNN (mutuo si es posible)
    W_alpha_vis = build_knn_graph_from_dense(sim_alpha, topk=min(10, TOPK), mutual=True)
    if W_alpha_vis.nnz == 0:
        W_alpha_vis = build_knn_graph_from_dense(sim_alpha, topk=min(12, TOPK + 2), mutual=False)

    # 2) Tamaños y colores de nodos a partir de Shannon
    sh = alpha_df['shannon'].astype(float).to_numpy()
    sh_min, sh_max = float(np.min(sh)), float(np.max(sh))
    sh_norm = (sh - sh_min) / (sh_max - sh_min + 1e-8)  # [0,1]
    node_sizes = 60 + 220 * sh_norm
    node_colors = sh_norm  # mapeo directo a colormap

    # 3) Si hay networkx disponible, úsalo con compatibilidad de versión
    use_nx = ('nx' in globals()) and (nx is not None)
    if use_nx:
        # Compatibilidad NX2/NX3
        _from_sparse = getattr(nx, "from_scipy_sparse_array",
                               getattr(nx, "from_scipy_sparse_matrix", None))
        if _from_sparse is None:
            raise AttributeError("NetworkX no ofrece from_scipy_sparse_* en esta versión.")
        G_vis = _from_sparse(W_alpha_vis)
        for (u, v, d) in G_vis.edges(data=True):
            d['weight'] = float(d.get('weight', 1.0))

        try:
            pos = nx.spring_layout(G_vis, seed=GLOBAL_SEED)
        except Exception:
            pos = nx.spectral_layout(G_vis)

        import matplotlib as mpl
        from matplotlib.collections import LineCollection

        edges = list(G_vis.edges(data=True))
        if edges:
            weights = np.array([e[2].get('weight', 1.0) for e in edges], dtype=float)
            w_min, w_max = float(np.min(weights)), float(np.max(weights))
            w_norm = (weights - w_min) / (w_max - w_min + 1e-12)
            segs = [ [pos[u], pos[v]] for u, v, _ in edges ]
            lc = LineCollection(segs, array=w_norm, cmap='viridis', linewidths=0.8, alpha=0.8)

        fig, ax = plt.subplots(figsize=(8.2, 8.2))
        sc = ax.scatter(
            [pos[i][0] for i in range(len(sh))],
            [pos[i][1] for i in range(len(sh))],
            s=node_sizes, c=node_colors, cmap='Blues',
            edgecolors='white', linewidths=0.3, alpha=0.95
        )

        # Aristas
        if edges:
            ax.add_collection(lc)
            cbar_e = fig.colorbar(lc, ax=ax, fraction=0.046, pad=0.02)
            cbar_e.set_label('Intensidad de enlace (similitud alfa normalizada)', rotation=90)

        # --- BARRA DE COLOR HORIZONTAL (nodos) ---
        cbar_n = fig.colorbar(sc, ax=ax, orientation='horizontal', fraction=0.05, pad=0.12)
        cbar_n.set_label('Diversidad de Shannon (normalizada)', rotation=0, labelpad=5)

        # Resaltar nodos aislados
        deg = dict(G_vis.degree())
        iso_idx = [i for i, d in deg.items() if d == 0]
        if len(iso_idx) > 0:
            ax.scatter(
                [pos[i][0] for i in iso_idx],
                [pos[i][1] for i in iso_idx],
                s=node_sizes[iso_idx]*1.05, facecolors='none', edgecolors='crimson', linewidths=1.0,
                label='Aislados'
            )
            ax.legend(loc='upper left', frameon=False)

        ax.set_title('Grafo kNN por similitud alfa\n(tamaño y color del nodo ∝ Shannon)')
        ax.set_axis_off()
        fig.tight_layout()
        plt.show()

    else:
        # --- Fallback sin networkx ---
        from scipy.sparse import csgraph
        from scipy.sparse.linalg import eigsh
        L = csgraph.laplacian(W_alpha_vis, normed=True)
        try:
            vals, vecs = eigsh(L, k=3, sigma=0.0, which='LM')
            idx = np.argsort(vals)
            coords = vecs[:, idx[1:3]]
            x, y = coords[:, 0], coords[:, 1]
        except Exception:
            rng = np.random.default_rng(GLOBAL_SEED)
            x = rng.normal(size=W_alpha_vis.shape[0])
            y = rng.normal(size=W_alpha_vis.shape[0])

        coo = W_alpha_vis.tocoo()
        w = coo.data.astype(float)
        if w.size > 0:
            w_min, w_max = float(np.min(w)), float(np.max(w))
            w_norm = (w - w_min) / (w_max - w_min + 1e-12)

        fig, ax = plt.subplots(figsize=(8.2, 8.2))
        if w.size > 0:
            from matplotlib.collections import LineCollection
            segs = [ [(x[u], y[u]), (x[v], y[v])] for u, v in zip(coo.row, coo.col) ]
            lc = LineCollection(segs, array=w_norm, cmap='plasma', linewidths=0.8, alpha=0.8)
            ax.add_collection(lc)
            cbar_e = fig.colorbar(lc, ax=ax, fraction=0.046, pad=0.02)
            cbar_e.set_label('Intensidad de enlace (similitud alfa normalizada)', rotation=90)

        sc = ax.scatter(x, y, s=node_sizes, c=node_colors, cmap='Blues',
                        edgecolors='white', linewidths=0.3, alpha=0.95)
        # --- BARRA DE COLOR HORIZONTAL ABAJO ---
        cbar_n = fig.colorbar(sc, ax=ax, orientation='horizontal', fraction=0.05, pad=0.12)
        cbar_n.set_label('Diversidad de Shannon (normalizada)', rotation=0, labelpad=5)

        deg = np.asarray(W_alpha_vis.getnnz(axis=1)).ravel()
        iso = np.where(deg == 0)[0]
        if iso.size > 0:
            ax.scatter(x[iso], y[iso], s=node_sizes[iso]*1.05, facecolors='none',
                       edgecolors='crimson', linewidths=1.0, label='Aislados')
            ax.legend(loc='upper left', frameon=False)

        ax.set_title('Grafo kNN por similitud alfa\n(tamaño y color del nodo ∝ Shannon)')
        ax.set_axis_off()
        fig.tight_layout()
        plt.show()

except Exception as e:
    print(f"No se pudo dibujar el grafo de similitud alfa: {e}")




In [ ]:

# Bloque 11: Estabilidad de clusterización (NMI)
from sklearn.metrics import normalized_mutual_info_score as nmi_score
import numpy as np
import matplotlib.pyplot as plt

Kg_vals_lite = np.geomspace(0.8, 1.6, 3)
alpha_vals_lite = np.linspace(0.12, 0.24, 3)
kuramoto_lite = kuramoto_defaults.copy()
kuramoto_lite.update({'T':800, 'burn':320})
phase_affinity_lite = phase_affinity_defaults.copy()
phase_affinity_lite.update({'stride':4, 'chunk_n':128})
labels_grid={}
for Kg in Kg_vals_lite:
    for alpha_val in alpha_vals_lite:
        try:
            phases_tmp = simulate_kuramoto(results['W_fused'], Kg=Kg, **kuramoto_lite)
            aff_tmp = phase_affinity(phases_tmp, **phase_affinity_lite)
            W_tmp = build_knn_graph_from_dense(aff_tmp, topk=8, mutual=False)
            lbls, Qtmp, method_tmp = cluster_graph(W_tmp, ['louvain','label_propagation','leiden'], seed=kuramoto_lite['seed'])
            labels_grid[(Kg, alpha_val)] = lbls
        except Exception as e:
            print(f"Error en rejilla NMI Kg={Kg}, alpha={alpha_val}: {e}")
            labels_grid[(Kg, alpha_val)] = np.zeros(results['shape_used'][1], dtype=int)

nmi_matrix=np.zeros((len(Kg_vals_lite), len(alpha_vals_lite)))
for i, Kg in enumerate(Kg_vals_lite):
    for j, alpha_val in enumerate(alpha_vals_lite):
        ref_labels = labels_grid[(Kg_vals_lite[0], alpha_vals_lite[0])]
        curr_labels = labels_grid[(Kg, alpha_val)]
        if len(set(ref_labels))==1 or len(set(curr_labels))==1:
            nmi=0.0
        else:
            nmi = nmi_score(ref_labels, curr_labels)
        nmi_matrix[i,j] = nmi
plt.figure(figsize=(5,4))
im=plt.imshow(nmi_matrix, origin='lower', cmap='viridis', aspect='auto')
plt.colorbar(im, label='NMI')
plt.xticks(range(len(alpha_vals_lite)), [f"{a:.2f}" for a in alpha_vals_lite])
plt.yticks(range(len(Kg_vals_lite)), [f"{k:.2f}" for k in Kg_vals_lite])
plt.xlabel('alpha')
plt.ylabel('K_g')
plt.title('Estabilidad de clusterización (NMI)')
plt.tight_layout()
plt.show()
